In [18]:
# %pip install -q sentence-transformers numpy pandas scikit-learn
%pip install -q psycopg2-binary sentence-transformers numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
model = SentenceTransformer("all-MiniLM-L6-v2")

products = [
    ("Organic Green Tea", "A refreshing organic green tea rich in antioxidants."),
    ("Black Coffee Beans", "Premium roasted coffee beans with a strong aroma."),
    ("Herbal Chamomile Tea", "Relaxing chamomile tea to reduce stress and improve sleep."),
    ("Espresso Coffee Powder", "Finely ground espresso powder for instant energy."),
    ("Lemon Ginger Tea", "Zesty lemon and ginger tea blend for immunity boost."),
    ("Meiji", "Baby milk good for infant"),
    ("Nido", "Milk for kids"),
]

# Generate embeddings
descriptions = [desc for _, desc in products]
embeddings = model.encode(descriptions, convert_to_numpy=True, normalize_embeddings=True)

In [34]:
import psycopg2
from psycopg2.extras import execute_values

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

records = [(name, desc, embedding.tolist()) for (name, desc), embedding in zip(products, embeddings)]

insert_query = """
INSERT INTO products (name, description, embedding)
VALUES %s
"""
execute_values(cursor, insert_query, records)
conn.commit()

print("✅ Data inserted successfully.")

✅ Data inserted successfully.


In [ ]:
query_text = "Lactogen: good baby milk"
query_vector = model.encode([query_text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

cursor.execute("""
    SELECT name, description, 1 - (embedding <=> %s::vector) AS similarity
    FROM products
    ORDER BY embedding <=> %s::vector
    LIMIT 3;
""", (query_vector, query_vector))

rows = cursor.fetchall()

print("\n🔍 Recommendations for:", query_text)
for name, desc, sim in rows:
    print(f"\n{name} ({sim:.3f})\n{desc}")


🔍 Recommendations for: Lactogen: good baby milk

Meiji (0.729)
Baby milk good for infant

Nido (0.646)
Milk for kids

Lemon Ginger Tea (0.332)
Zesty lemon and ginger tea blend for immunity boost.
